In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

sys.path.append('/coc/scratch/anarayanan68/mint/')

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, initializers, regularizers

from mint.core import fact_model, base_model_util, base_models, primitive_models, model_builder
from mint.utils import inputs_util, config_util

In [4]:
config_path = '/coc/scratch/anarayanan68/mint/configs/audio_based_blending__embed_cond_vary_num_prims.config'
enc_config_path = '/coc/scratch/anarayanan68/mint/configs/audio_based_blending__embed_cond_vary_num_prims-enc_config.yml'

In [5]:
expt_root = '/coc/scratch/anarayanan68/mint/_expts/audio_based_blending__primitive_loss/10prims'

checkpoint_dir = os.path.join(expt_root, 'checkpoints/')

In [6]:
configs = config_util.get_configs_from_pipeline_file(config_path)
model_config = configs['model']
eval_config = configs['eval_config']
eval_dataset_config = configs['eval_dataset']

In [10]:
enc_config_yaml = config_util.read_yaml_config(enc_config_path)
num_primitives = 10

In [11]:
# Model build & restore

model = model_builder.build(model_config, is_training=False,
    num_primitives=num_primitives, encoder_config_yaml=enc_config_yaml, dataset_config=eval_dataset_config)

checkpoint_manager=tf.train.CheckpointManager(
    tf.train.Checkpoint(model=model),
    directory=checkpoint_dir,
    max_to_keep=None)

checkpoint_manager.restore_or_initialize()

'/coc/scratch/anarayanan68/mint/_expts/audio_based_blending__primitive_loss/10prims/checkpoints/ckpt-9999'

In [12]:
all_primitives = model.blend_vec_to_seq(np.eye(num_primitives))
all_primitives

<tf.Tensor: shape=(10, 120, 147), dtype=float32, numpy=
array([[[ 3.3190239e-02,  1.7497697e+00, -3.0541164e-01, ...,
          5.8972882e-03,  9.8340136e-01,  2.1387385e-03],
        [ 3.2793544e-02,  1.7505261e+00, -3.0513600e-01, ...,
          5.7240548e-03,  9.8340166e-01,  2.0985205e-03],
        [ 3.2733031e-02,  1.7520490e+00, -3.0426842e-01, ...,
          6.2224963e-03,  9.8339880e-01,  2.2611252e-03],
        ...,
        [ 7.5281486e-03,  7.5208819e-03,  7.5243837e-03, ...,
          7.5220419e-03,  7.5246431e-03,  7.5284904e-03],
        [ 7.5279577e-03,  7.5192880e-03,  7.5247013e-03, ...,
          7.5224005e-03,  7.5255777e-03,  7.5287782e-03],
        [ 7.5282068e-03,  7.5205187e-03,  7.5248196e-03, ...,
          7.5230682e-03,  7.5249332e-03,  7.5286287e-03]],

       [[ 8.2359957e-03,  2.2788037e-02, -8.3739767e-03, ...,
         -5.2015902e-04,  2.0457603e-02, -2.4870996e-04],
        [ 7.0737228e-03,  2.2878394e-02, -8.3125364e-03, ...,
         -6.4258889e-04,  2

In [13]:
save_dir = os.path.join(expt_root, 'primitives')
os.makedirs(save_dir, exist_ok=True)
for i in range(num_primitives):
    np.save(os.path.join(save_dir, f"PRIMITIVE--{i+1}_of_{num_primitives}.npy"), all_primitives[i])